# Detect people loitering 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uclasystem/VQPy/blob/main/examples/loitering/demo.ipynb)


## Introduction

Loitering is the activity of remaining in an area for no obvious reason. In the example [video](https://youtu.be/EuLMrUFNRxQ), we consider the region inside the garage private, and tries to detect people staying within that region for a prolonged time.

## Environment setup

Please download video from [here](https://youtu.be/EuLMrUFNRxQ) and place it in the same directory as this notebook, aside from running the cell below.

In [ ]:
# install YOLOX
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
# download YOLOX pretrained model
!wget -bqc https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.pth
!cd YOLOX && pip3 install .
# download VQPy, move vqpy/ to root directory for import
!git clone https://github.com/uclasystem/VQPy.git
!mv VQPy/vqpy ./
# install VQPy's dependencies
!pip install lap cython_bbox shapely
import vqpy

## Composing the query

### VObj definition

Since we are interested in people, first define a `Person` class. In this example, we will only be using the coordinates of each person, which are all built-in to VObj, so no extra property needs to be added.

In [ ]:
class Person(vqpy.VObjBase):
    pass

With the defined VObj type, we can filter all VObjs that's identified to be a person with:

```python
"__class__": lambda x: x == Person
```

### Query definition

#### Filtering VObjs

Trying to find people that stays in the region of interest, we filter on the coordinate of a person's base of support. This can be approximated to a built-in property of VObj: `bottom_center`, the point located at the center of the lower edge of VObj's bounding box.

Knowing where the person is, the next step is to determine if the coordinate is within the region of interest (i.e. the garage), marked by the red polygon shown below.

<img src="./demo.assets/region.png" alt="region of interest" style="zoom: 30%;" />

`vqpy.utils.within_regions([REGION, ...])` accepts a coordinate and returns `True` if it's in any of the `REGION`s.

The polygon is outlined by its 5 vertices:

In [ ]:
REGION = [(550, 550), (1162, 400), (1720, 720), (1430, 1072), (600, 1073)]

The predicate to filter VObjs in the region is then:

```python
"bottom_center": vqpy.utils.within_regions([REGION])
```

Since we wish to find person staying in the region for some duration, `vqpy.utils.continuing` could wrap the above predicate and specify the minimum time it should be satisfied.

`duration=10` sets the time threshold to 10 seconds; `name="in_roi"` can be used later to retrieve time periods of condition being satisfied.

Predicate used to filter on `bottom_center` becomes:

```python
"bottom_center": vqpy.utils.continuing(
    condition=vqpy.utils.within_regions([REGION]),
    duration=10, name="in_roi"
)
```

Combining everything we have so far, the filter condition of the query should be:

In [ ]:
filter_cons = {
    "__class__": lambda x: x == Person,
    "bottom_center": vqpy.utils.continuing(
        condition=vqpy.utils.within_regions([REGION]), duration=10, name="in_roi"
    ),
}

#### Selecting VObjs' properties for output

For each person, we can output its tracking id, coordinate, and duration of staying in the region:

- tracking id can be selected with `track_id`
- outputting coordinate needs to select `coordinate` then serialize the ndarray to string with post-processing function `lambda x: str(x)`
- `in_roi_periods` can be used to access list of time periods (in seconds) of VObjs satisfying the condition

	this name comes from appending `"_periods"` to `name` defined in `vqpy.utils.continuing`

The select condition of the query could be:

In [ ]:
select_cons = {
    "track_id": None,
    "coordinate": lambda x: str(x),  # convert to string for JSON serialization
    # name in vqpy.continuing + '_periods' can be used in select_cons.
    "in_roi_periods": None,
}

### The query

Inheriting `vqpy.QueryBase`, we can compose a query using the `filter_cons, select_cons` created earlier:

In [ ]:
class People_loitering_query(vqpy.QueryBase):
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        return vqpy.VObjConstraint(
            filter_cons, select_cons, filename="loitering"
        )

## Running the query

In [ ]:
video_path = "./loitering.mp4"
save_folder = "./vqpy_outputs"
detector_model_dir = "./"

In [ ]:
vqpy.launch(
        cls_name=vqpy.COCO_CLASSES,
        cls_type={"person": Person},
        tasks=[People_loitering_query()],
        video_path=video_path,
        save_folder=save_folder,
        detector_name="yolox",
        detector_model_dir=detector_model_dir,
    )

Note that:

- we specified the class number in output of the object detector should be interpreted as described in `vqpy.COCO_CLASSES
- then mapped `person` in COCO classes to `Person` VObj defined above.

## Expected result

The output should be in `./vqpy_outputs/loitering_loitering_yolox.json`.

One entry is created for each frame that has filter condition satisfied.

The 765th frame as an example:

```json
{
    "frame_id": 765,
    "data": [
      {
        "track_id": 606,
        "coordinate": "[1426.5312   492.42188]",
        "in_roi_periods": [[40, 50]]
      }
    ]
}
```

Marking person's `coordinate` with a red circle, this is

<img src="./demo.assets/marked.png" alt="with coordinate marked" style="zoom: 30%;" />

Value of `in_roi_periods` indicates that the person has been in the region in time period 00:40-00:50.